# Data walking through

In [9]:
import pandas as pd 
import numpy as np

# read csv and assign columns
columns = ["personId", "productId"]
df = pd.read_csv("dataset/ratebeer/wish.csv", header=None)
df.columns = columns
# get number of products
number_of_product = len(np.unique(df["productId"]))
# get number of person
number_of_person = len(np.unique(df["personId"]))
df = df.groupby("personId").aggregate(lambda x: list(x))

In [10]:
print("Number of person:", number_of_person)
print("Number of product:", number_of_product)
# first samples
df.head()

Number of person: 1953
Number of product: 21654


,productId
personId,
39,"[47436, 1147, 64658, 9448, 12227, 10609, 6890,..."
63,"[36407, 53647, 36624]"
69,"[15017, 2491, 6924, 9193, 19633, 13730, 64975,..."
154,"[49774, 28173, 15017, 24350, 565, 22339, 17385..."
181,"[10524, 30591, 7183, 38426, 22537, 48679, 2912..."


In [18]:
# 'transactions' is now temporary variable
transactions = [row["productId"] for index, row in df.iterrows()]

In [33]:
from mlxtend.preprocessing import TransactionEncoder

transaction_encoder = TransactionEncoder()
wish_matrix = transaction_encoder.fit_transform(transactions).astype("int")
wish_df = pd.DataFrame(wish_matrix, columns=transaction_encoder.columns_)

In [34]:
wish_df.head()

,1,2,3,9,10,13,14,15,16,17,25,29,31,48,49,50,55,59,64,67,68,70,71,72,76,85,87,94,96,97,103,105,106,107,109,110,111,112,114,115,...,387759,387781,387782,387885,388103,388104,388332,388571,388638,388647,388656,388749,388781,388846,388876,388996,389013,389025,389104,389220,389221,389261,389306,389721,389952,390094,390176,390251,390260,390294,390987,391005,391163,391340,391802,392092,392165,392402,392478,393042
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Transaction-based wish prediction using apriori

In [46]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(wish_df, min_support=0.1)
rules = association_rules(frequent_itemsets, metric="lift")
# Something FAILED
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


# User-based prediction using cosine similarity

In [48]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([wish_df[1]], [wish_df[25]])

array([[0.]])